**Importar librerías**

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
#!pip install openpyxl


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from scipy.stats import spearmanr
from sklearn.model_selection import KFold

**Definir parámetros**

In [55]:
# Parámetros globales
window_days = 1      #establecer diferentes ventanas [1,2,3,5]
rows, cols, channels = 95, 68, 1 # Reducción en la resolución de las imágenes
n_iteraciones = 1 #aumentar hasta (10,20,30) después de las pruebas

**Cargar y escalar datos**

In [56]:
# Ruta del archivo de datos
data_path = '/content/drive/MyDrive/DL_ALGAS/Datos_dias.xlsx'
data = pd.read_excel(data_path)

# Selección de columnas de entrada y salida
input_features = ['Irradiancia', 'NO3', 'TEMP', 'pH', 'CO2 Gas']
output_feature = 'biomasa'

# Aplicar escaladores
#scaler = StandarScaler()
#scaler = RobustScaler()
scaler = MinMaxScaler()
data[input_features] = scaler.fit_transform(data[input_features])
data[output_feature] = scaler.fit_transform(data[[output_feature]])


**Cargar imágenes**

In [57]:
# Cargar imágenes
image_dir = '/content/drive/MyDrive/DL_ALGAS/Datos_dias'
all_images = [img_to_array(load_img(os.path.join(image_dir, img_name), color_mode='grayscale', target_size=(rows, cols))) / 255.0 for img_name in sorted(os.listdir(image_dir))]
all_images = np.array(all_images)
print("Total de imágenes cargadas:", all_images.shape)

Total de imágenes cargadas: (31, 95, 68, 1)


**Definir función de entrenamiento del modelo**

In [58]:
# Definir función para entrenar modelo
def train_model(data, images, window_days, n_iteraciones):
    metrics_summary = {'R2': [], 'MSE': [], 'RMSE': [], 'MAE': [], 'Spearman': [], 'MAPE': []}
    y_vals_all, y_preds_all = [], []
    history_per_iteration = []
    img_sequences = []
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    for iteration in range(n_iteraciones):
        img_sequences, target_sequences, feature_sequences = [], [], []
        for i in range(len(images) - window_days + 1):
            img_sequences.append(images[i:i + window_days])
            target_sequences.append(data[output_feature].iloc[i + window_days - 1])
            feature_sequences.append(data[input_features].iloc[i + window_days - 1].values)

        img_sequences = np.array(img_sequences).reshape((-1, window_days, rows, cols, channels))
        target_sequences = np.array(target_sequences)
        feature_sequences = np.array(feature_sequences)

        for train_index, val_index in kfold.split(img_sequences):
            img_train, img_val = img_sequences[train_index], img_sequences[val_index]
            y_train, y_val = target_sequences[train_index], target_sequences[val_index]
            features_train, features_val = feature_sequences[train_index], feature_sequences[val_index]

            # Definir el modelo CNN-LSTM con entradas adicionales
            inp_image = Input(shape=(window_days, rows, cols, channels))
            x = ConvLSTM2D(32, (3, 3), padding="same", activation="tanh", return_sequences=True)(inp_image)
            x = BatchNormalization()(x)
            x = ConvLSTM2D(64, (3, 3), padding="same", activation="tanh", return_sequences=True)(x)
            x = BatchNormalization()(x)
            x = ConvLSTM2D(128, (3, 3), padding="same", activation="tanh", return_sequences=False)(x)
            x = BatchNormalization()(x)
            x = Flatten()(x)

            inp_features = Input(shape=(len(input_features),))
            combined = Concatenate()([x, inp_features])

            combined = Dense(128, activation='relu')(combined)
            combined = Dropout(0.7)(combined)
            combined = Dense(128, activation='relu')(combined)
            combined = Dropout(0.3)(combined)
            output = Dense(1, activation='linear')(combined)

            model = Model(inputs=[inp_image, inp_features], outputs=output)
            model.compile(optimizer=Adam(learning_rate=1e-4), loss="mse", metrics=["mae", "mse"])

            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            history = model.fit([img_train, features_train], y_train, validation_data=([img_val, features_val], y_val), epochs=100, batch_size=16, callbacks=[early_stop])
            history_per_iteration.append(history.history)

            y_pred = model.predict([img_val, features_val]).flatten()
            metrics_summary['R2'].append(r2_score(y_val, y_pred))
            metrics_summary['MSE'].append(mean_squared_error(y_val, y_pred))
            metrics_summary['RMSE'].append(np.sqrt(mean_squared_error(y_val, y_pred)))
            metrics_summary['MAE'].append(mean_absolute_error(y_val, y_pred))
            metrics_summary['Spearman'].append(spearmanr(y_val, y_pred).correlation)
            metrics_summary['MAPE'].append(mean_absolute_percentage_error(y_val, y_pred))

            y_vals_all.append(y_val)
            y_preds_all.append(y_pred)

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}
    return {
        'model': model,
        'history': history_per_iteration,
        'metrics': avg_metrics,
        'y_val': np.concatenate(y_vals_all),
        'y_pred': np.concatenate(y_preds_all),
        'img_val': img_val,  # Añadir img_val para análisis de importancia
        'features_val': features_val  # Añadir features_val para análisis de importancia
    }

**Previsualización de resultados**

In [ ]:
# Resultados del entrenamiento
result = train_model(data, all_images, window_days, n_iteraciones)

# Previsualización de los resultados
print("\nResultados de la métrica promedio:")
for metric, value in result['metrics'].items():
    print(f"{metric}: {value}")

Epoch 1/100
2/2 [==============================] - 9s 2s/step - loss: 70.8881 - mae: 4.3106 - mse: 70.8881 - val_loss: 0.4686 - val_mae: 0.6590 - val_mse: 0.4686
Epoch 2/100
2/2 [==============================] - 2s 1s/step - loss: 360.6135 - mae: 14.1287 - mse: 360.6135 - val_loss: 0.4680 - val_mae: 0.6584 - val_mse: 0.4680
Epoch 3/100
2/2 [==============================] - 2s 1s/step - loss: 478.5487 - mae: 16.7234 - mse: 478.5487 - val_loss: 0.4647 - val_mae: 0.6560 - val_mse: 0.4647
Epoch 4/100
2/2 [==============================] - 2s 1s/step - loss: 239.0832 - mae: 13.2201 - mse: 239.0832 - val_loss: 0.4596 - val_mae: 0.6521 - val_mse: 0.4596
Epoch 5/100
2/2 [==============================] - 2s 1s/step - loss: 123.2621 - mae: 8.6409 - mse: 123.2621 - val_loss: 0.4520 - val_mae: 0.6463 - val_mse: 0.4520
Epoch 6/100
2/2 [==============================] - 2s 1s/step - loss: 194.7638 - mae: 11.0629 - mse: 194.7638 - val_loss: 0.4442 - val_mae: 0.6402 - val_mse: 0.4442
Epoch 7/100
2/

**Exportar resultados**

In [ ]:
# Exportar métricas a Excel
excel_path = '/content/drive/MyDrive/DL_ALGAS/Resultados_dias.xlsx'
with pd.ExcelWriter(excel_path) as writer:
    df_metrics = pd.DataFrame([result['metrics']])
    df_metrics.to_excel(writer, sheet_name=f'Ventana_{window_days}d', index=False)
print(f'Resumen de métricas guardado en {excel_path}')

**Generar gráficos**

In [ ]:
# Gráficos de resultados
y_val = result['y_val']
y_pred = result['y_pred']
history = result['history']

In [ ]:
# Serie Temporal de Predicción vs Valores Reales
plt.figure(figsize=(12, 6))
plt.plot(y_val, label='Valores Reales', color='blue')
plt.plot(y_pred, label='Valores Predichos', color='orange', linestyle='--')
plt.title(f'Predicción vs Valores Reales - Ventana {window_days} días')
plt.xlabel('Índice')
plt.ylabel('Biomasa')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de Dispersión de Predicción vs Real
plt.figure(figsize=(10, 6))
scatter = plt.scatter(y_val, y_pred, c=np.abs(y_val - y_pred), cmap='viridis', s=70, alpha=0.9, edgecolor='black')
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=2)
plt.title(f'Valores Reales vs. Predichos - Ventana {window_days} días')
plt.xlabel('Valores Reales de Biomasa')
plt.ylabel('Valores Predichos de Biomasa')
cbar = plt.colorbar(scatter)
cbar.set_label('Error Absoluto')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de Error Residual
residuals = y_val - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_val, residuals, alpha=0.6, color="purple")
plt.axhline(0, linestyle='--', color='black')
plt.title(f'Errores Residuales - Ventana {window_days} días')
plt.xlabel('Valores Reales de Biomasa')
plt.ylabel('Error Residual')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Histograma de Errores
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=20, color="darkblue", edgecolor='black')
plt.title(f'Distribución de Errores - Ventana {window_days} días')
plt.xlabel('Error Residual')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Curva de Aprendizaje
plt.figure(figsize=(10, 6))
for fold_history in history:
    # Verificar si ya existe una leyenda
    legend = plt.gca().get_legend()

    # Agregar etiquetas solo si aún no existen
    if legend is None or 'Entrenamiento' not in [text.get_text() for text in legend.get_texts()]:
        plt.plot(fold_history['loss'], color='blue', alpha=0.3, label='Entrenamiento')
    else:
        plt.plot(fold_history['loss'], color='blue', alpha=0.3)

    if legend is None or 'Validación' not in [text.get_text() for text in legend.get_texts()]:
        plt.plot(fold_history['val_loss'], color='orange', alpha=0.3, label='Validación')
    else:
        plt.plot(fold_history['val_loss'], color='orange', alpha=0.3)

plt.title(f'Curva de Aprendizaje - Ventana {window_days} días')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# Gráfico de Importancia de Variables (Análisis de Sensibilidad)
importances = []
X = data[input_features].values  # Usar las columnas de características de entrada
X_val = X[:len(y_val)]  # Ajustar el tamaño para el conjunto de validación

for feature_idx in range(X.shape[1]):
    X_temp = X_val.copy()
    X_temp[:, feature_idx] = np.mean(X[:, feature_idx])  # Perturbar la variable
    perturbed_pred = result['model'].predict([img_val, X_temp]).flatten()
    importance = mean_absolute_error(y_val, perturbed_pred)
    importances.append(importance)

# Gráfico de Importancia de Características
plt.figure(figsize=(10, 6))
sns.barplot(x=input_features, y=importances, palette="viridis")
plt.title(f'Importancia de Variables - Ventana {window_days} días')
plt.xlabel('Variables Ambientales')
plt.ylabel('Importancia (MAE)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap de Correlación entre Variables
plt.figure(figsize=(10, 8))
sns.heatmap(data[input_features + [output_feature]].corr(), annot=True, cmap="YlGnBu")
plt.title(f"Heatmap de Correlación entre Variables")
plt.tight_layout()
plt.show()